In [1]:
!pip install -U discord.py
!pip install langchain
!pip install -qU langchain-groq
!pip install python-dotenv
!pip install nest_asyncio
!pip install youtube-transcript-api

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.4/396.4 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.5/290.5 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.4 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 2.6 MB/s eta 0:00:00


In [2]:
import langchain
import os
from langchain_groq import ChatGroq
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
import discord
from discord.ext import commands,tasks
from discord import File
import asyncio
from youtube_transcript_api import YouTubeTranscriptApi
from googleapiclient.discovery import build

In [3]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"

In [4]:
from google.colab import files

# Upload the .env file
uploaded = files.upload()

# Install python-dotenv package
!pip install python-dotenv

import os
from dotenv import load_dotenv

# Load the .env file
load_dotenv('myenv.env')

# Access the API keys
os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGCHAIN_API_KEY')
os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')
os.environ['YOUTUBE_API_KEY'] = os.getenv('YOUTUBE_API_KEY')
BOT_TOKEN = "ENTER YOUR BOT TOKEN"

Saving myenv.env to myenv.env


In [5]:
llm = ChatGroq(model='llama3-8b-8192')

In [6]:
class YouTubeSummarizer:
    def __init__(self, custom_url, discord_channel_name):
        self.custom_url = custom_url
        self.discord_channel_name = discord_channel_name
        self.youtube = build('youtube', 'v3', developerKey=os.environ['YOUTUBE_API_KEY'])
        self.llm = llm
        self.transcript_summary_prompt = PromptTemplate(
            input_variables=["transcript"],
            template="""
            Summarize the following transcript into concise bullet points:

            {transcript}

            Summary:
            -
            """
        )

    def get_channel_id_from_custom_url(self):
        request = self.youtube.search().list(
            part='snippet',
            q=self.custom_url,
            type='channel'
        )
        response = request.execute()
        channel_id = response['items'][0]['snippet']['channelId']
        return channel_id

    def get_latest_video_id(self, channel_id):
        request = self.youtube.search().list(
            part='snippet',
            channelId=channel_id,
            maxResults=1,
            order='date'
        )
        response = request.execute()
        latest_video_id = response['items'][0]['id']['videoId']
        return latest_video_id

    def fetch_transcript(self, video_id):
        transcript = YouTubeTranscriptApi.get_transcript(video_id)
        full_transcript = " ".join([entry['text'] for entry in transcript])
        return full_transcript

    async def summarize_transcript(self, transcript):
        summary = self.transcript_summary_prompt | self.llm | StrOutputParser()
        summary_text = summary.invoke({"transcript": transcript})
        return summary_text

    async def post_summary_to_discord(self, summary, bot):
        for guild in bot.guilds:
            for channel in guild.text_channels:
                if channel.name == self.discord_channel_name:
                    await channel.send(summary)
                    break

    async def summarize_latest_video(self, bot):
        channel_id = self.get_channel_id_from_custom_url()
        latest_video_id = self.get_latest_video_id(channel_id)
        transcript = self.fetch_transcript(latest_video_id)
        summary = await self.summarize_transcript(transcript)
        await self.post_summary_to_discord(summary, bot)

    async def periodic_summarization(self, bot, interval):
        while True:
            await self.summarize_latest_video(bot)
            await asyncio.sleep(interval)

In [7]:
class ModerationChain():
    def __init__(self):
        super().__init__()
        self.llm = ChatGroq(model="llama3-8b-8192")

    def process(self, message_content: str) -> bool:

        goat_prompt = PromptTemplate(
            input_variables=["message_content"],
            template="""
            You are an avid Lionel Messi fan. Analyze the following message and determine the tone of the message.
            If the tone of the message includes criticism, vulgarity, racial slurs, or mockery towards Lionel Messi,
            return "1". Otherwise, return "0". If the context of the message is unclear or inconclusive, return "0".
            If the message is too short to understand the context, return "0". if the message is not in english, return "0".

            Message: {message_content}

            Your output should be in the following format:
            0 or 1

            VERY IMPORTANT: This format should be stricly followed. No other text or explanation is allowed. DONOT add any other text or explanation. If the message is not in english, return "0".
            """
        )

        parser = StrOutputParser()
        chain = goat_prompt | self.llm | parser

        answer = chain.invoke({"message_content": message_content})

        # Ensure the output is either "0" or "1"
        try:
            result = int(answer.strip())
            if result not in [0, 1]:
                raise ValueError("Output not 0 or 1")
        except ValueError:
            result = int(0)  # Default to not deleting the message if the output is unexpected

        return answer


In [8]:
intents = discord.Intents.default()
intents.messages = True
intents.message_content = True  # This line is necessary to access message content

bot = commands.Bot(command_prefix='#', intents=intents)

In [9]:
quote_prompt = PromptTemplate(
    template="""
    Give a Real quote about Lionel Messi, said by football legends. Make sure that the quote is in english and is not fake.

    your output should be in the following format:
    'They tell me that all men are equal in God’s eyes, this player makes you seriously think about those words.' - soccer commentator Ray Hudson

    dont deviate from this format. DONOT write Here's a real quote about Lionel Messi, said by a football legend: or anything else at the beginning.
    """,
    )

youtube_summarizer = YouTubeSummarizer(custom_url="FabrizioRomanoYT", discord_channel_name="football")

@bot.event
async def on_ready():
    print(f'Logged in as {bot.user}')

    image_url = "https://fifpro.org/media/fhmfhvkx/messi-world-cup.jpg?rxy=0.48356841796117644,0.31512414378031967&width=1000&height=640&rnd=133210253587130000"
    quote = quote_prompt | llm | StrOutputParser()
    # Create an embed with the image and quote
    quote_text = quote.invoke({})
    embed = discord.Embed(description=quote_text)
    embed.set_image(url=image_url)
    # Send the embed to the 'general' channel
    for guild in bot.guilds:
        for channel in guild.text_channels:
            if channel.name == 'general':
                await channel.send(embed=embed)
                break

    bot.loop.create_task(youtube_summarizer.periodic_summarization(bot, 3 * 60 * 60))


In [10]:
# Initialize the LangChain chain
mod = ModerationChain()
@bot.command()
async def delete(ctx, message_id: int):
    try:
        message = await ctx.channel.fetch_message(message_id)
        await message.delete()
        await ctx.send(f'Message {message_id} deleted.')
    except discord.NotFound:
        await ctx.send(f'Message {message_id} not found.')
    except discord.Forbidden:
        await ctx.send('I do not have permission to delete messages.')
    except discord.HTTPException as e:
        await ctx.send(f'Failed to delete message: {e}')

@bot.event
async def on_message(message):
    if message.author == bot.user:
        return

    if isinstance(message.channel, discord.DMChannel):
        return  # Ignore messages from DMs
    # Use the LangChain chain to decide whether to delete the message
    should_delete = mod.process(message.content)

    if should_delete == '1':
        await message.delete()
        await message.channel.send(f'Message deleted because messi is GOAT.')

    await bot.process_commands(message)

In [11]:
# Keep track of scores
user_scores = {}

# Track questions to avoid repetition
asked_questions = set()

# Function to generate a quiz question
async def generate_question():
    topic = "Lionel Messi"
    while True:
        prompt = f"Generate a random question about {topic}."
        question = llm.invoke(prompt)
        question = question.content.strip()
        if question not in asked_questions:
            asked_questions.add(question)
            return question

# Function to verify the answer
async def verify_answer(question, user_answer):
    prompt = (
        f"Question: {question}\n"
        f"User Answer: {user_answer}\n"
        "Please respond with 'Correct' or 'Incorrect'. "
        "Then, on a new line, provide the correct answer in the format: Correct answer: <answer>."
    )
    feedback = llm.invoke(prompt).content.strip()

    # Extract whether the answer is correct and the correct answer
    is_correct = "Correct" in feedback.split('\n')[0]
    correct_answer = feedback.split("Correct answer: ")[-1].strip()  # Assuming the model provides this format

    return is_correct, correct_answer

@bot.command()
async def quiz(ctx):
    user = ctx.author
    score = user_scores.get(user, 0)

    # Start the quiz session with 5 questions
    for i in range(5):
        question = await generate_question()
        await ctx.send(f"Question {i + 1}: {question}\nYou have 15 seconds to answer!")

        def check(m):
            return m.author == user and m.channel == ctx.channel

        try:
            # Wait for user's response with a 15-second timeout
            msg = await bot.wait_for('message', timeout=15.0, check=check)

            # Verify the answer
            is_correct = await verify_answer(question, msg.content)
            if is_correct:
                score += 1
                await ctx.send(f"Correct! Your score is now {score}.")
            else:
                await ctx.send(f"Incorrect. Your score remains {score}.")
        except asyncio.TimeoutError:
            await ctx.send(f"Time's up! The correct answer was not provided.")

    # Update the user's score
    user_scores[user] = score

    # Send the final score after 5 questions
    await ctx.send(f"Quiz complete! Your final score is {score} out of 5.")

@bot.command()
async def leaderboard(ctx):
    if user_scores:
        leaderboard = sorted(user_scores.items(), key=lambda x: x[1], reverse=True)
        leaderboard_message = "Leaderboard:\n"
        for i, (user, score) in enumerate(leaderboard, start=1):
            leaderboard_message += f"{i}. {user.display_name}: {score} points\n"
        await ctx.send(leaderboard_message)
    else:
        await ctx.send("No scores yet! Start a quiz with #quiz.")

In [12]:
import nest_asyncio
nest_asyncio.apply()
if __name__ == "__main__":
    bot.run(BOT_TOKEN)
else:
    # In an interactive environment, use a different method to run the bot
    async def run_bot():
        await bot.start(BOT_TOKEN)

    loop = asyncio.get_event_loop()
    loop.create_task(run_bot())
    loop.run_forever()

2024-09-10 15:21:49 INFO     discord.client logging in using static token
INFO:discord.client:logging in using static token
2024-09-10 15:21:49 INFO     discord.gateway Shard ID None has connected to Gateway (Session ID: c9aab56b39c794c3cb5fe28eba57d672).
INFO:discord.gateway:Shard ID None has connected to Gateway (Session ID: c9aab56b39c794c3cb5fe28eba57d672).


Logged in as GOATbot#3293
